In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, datediff, count

In [0]:
patients = spark.read.format("delta").load("/Volumes/workspace/synthea/synthea_datasets/patients_silver_delta")
encounters = spark.read.format("delta").load("/Volumes/workspace/synthea/synthea_datasets/encounters_silver_delta")
medications = spark.read.format("delta").load("/Volumes/workspace/synthea/synthea_datasets/medications_silver_delta")
conditions = spark.read.format("delta").load("/Volumes/workspace/synthea/synthea_datasets/conditions_silver_delta")

In [0]:
journey = (
    encounters.join(patients, "id", "inner")
              .join(conditions, "patient", "left")
              .join(medications, "patient", "left")
)
display(journey)

In [0]:
journey = (
    encounters.join(patients, patients.id == encounters.patient, "inner")
              .join(conditions, conditions.patient == patients.id, "left")
              .join(medications, medications.patient == patients.id, "left")
)
display(journey)

In [0]:
journey_features = (
    journey.groupBy("patient")
    .agg(
        min("date").alias("first_diagnosis_date"),
        min("start_date").alias("first_medication_date"),
        count("encounter").alias("total_encounters")
    )
    .withColumn("time_to_medication_start",
                datediff(col("first_medication_date"), col("first_diagnosis_date")))
)
display(journey_features)

